# Get full page screenshots from archived web pages

<p class="alert alert-warning">Work in progress – this is an experimental tool, it should work, but you never know...<p>

In [ ]:
%%capture
from selenium import webdriver
import requests
import selenium
from PIL import Image
import PIL
import io
import base64
import time
import re
import arrow
from slugify import slugify
from webdriverdownloader import GeckoDriverDownloader
from pathlib import Path
from IPython.display import display, HTML
from urllib.parse import urlparse
import ipywidgets as widgets

gdd = GeckoDriverDownloader()
geckodriver = gdd.download_and_install("v0.26.0")[1]

In [ ]:
%%capture
%%javascript
// This is necessary in Jupyter notebook to stop the output area folding up
// Will give an error in Jupyter Lab
IPython.OutputArea.prototype._should_scroll = function(lines) {return false}

In [ ]:
TIMEGATES = {
    'nla': 'https://web.archive.org.au/awa/',
    'nlnz': 'https://ndhadeliver.natlib.govt.nz/webarchive/wayback/',
    'bl': 'https://www.webarchive.org.uk/wayback/archive/',
    'ia': 'https://web.archive.org/web/'
}

pywb = ['webarchive.org.uk', 'web.archive.org.au', 'webarchive.nla.gov.au']
wayback = ['ndhadeliver.natlib.govt.nz', 'web.archive.org']

html_output = []

def format_date_for_headers(iso_date, tz):
    '''
    Convert an ISO date (YYYY-MM-DD) to a datetime at noon in the specified timezone.
    Convert the datetime to UTC and format as required by Accet-Datetime headers:
    eg Fri, 23 Mar 2007 01:00:00 GMT
    '''
    local = arrow.get(f'{iso_date} 12:00:00 {tz}', 'YYYY-MM-DD HH:mm:ss ZZZ')
    gmt = local.to('utc')
    return f'{gmt.format("ddd, DD MMM YYYY HH:mm:ss")} GMT'

def format_date_from_timestamp(url):
    timestamp = re.search(r'/(\d{14})(?:if_|mp_)*/', url).group(1)
    return arrow.get(timestamp, 'YYYYMMDDHHmmss').format('d MMMM YYYY')

def parse_links_from_headers(headers):
    '''
    Extract original, timegate, timemap, and memento links from 'Link' header.
    '''
    memento_links = {}
    try:
        links = re.findall(r'<(.*?)>; rel="(original|timegate|timemap|memento|first memento|prev memento|next memento|last memento)"', headers['Link'])
    except KeyError:
        # print(headers)
        return None
    else:
        for url, url_type in links:
            memento_links[url_type] = url
        return memento_links

def query_timegate(timegate, url, date=None, tz='Australia/Canberra'):
    headers = {}
    if date:
        formatted_date = format_date_for_headers(date, tz)
        headers['Accept-Datetime'] = formatted_date
    # BL & NLNZ don't seem to default to latest date if no date supplied
    elif not date and timegate in ['bl', 'nlnz']:
        formatted_date = format_date_for_headers(arrow.utcnow().format('YYYY-MM-DD'), tz)
        headers['Accept-Datetime'] = formatted_date
    # Note that you don't get a timegate response if you leave off the trailing slash, but extras don't hurt!
    tg_url = f'{TIMEGATES[timegate]}{url}/' if not url.endswith('/') else f'{TIMEGATES[timegate]}{url}'
    # print(tg_url)
    # IA doesn't work with head, others don't work with get...
    if timegate == 'ia':
        response = requests.get(tg_url, headers=headers)
    else:
        response = requests.head(tg_url, headers=headers)
    return parse_links_from_headers(response.headers)

def get_memento():
    links = query_timegate(repository.value, target_url.value, target_date.value)
    # NLNZ doesn't always seem to return a Memento, so we'll build in some fuzziness
    if links:
        if 'memento' in links:
            memento = links['memento']
        elif 'prev memento' in links:
            memento = links['prev memento']
        elif 'next memento' in links:
            memento = links['next memento']
        elif 'last memento' in links:
            memento = links['last memento']
    else:
        memento = None
    return memento
    
def get_full_page_screenshot(url, save_width=200):
    '''
    Gets a full page screenshot of the supplied url.
    By default resizes the screenshot to a maximum width of 200px.
    Provide a 'save_width' value to change this.
    
    NOTE the webdriver sometimes fails for unknown reasons. Just try again.
    '''
    global html_output
    domain = urlparse(url)[1].replace('www.', '')
    # NZ and IA inject content into the page, so we use if_ to get the original page (with rewritten urls)
    if domain in wayback and 'if_' not in url:
        url = re.sub(r'/(\d{14})/http', r'/\1if_/http', url)
    try:
        date_str, site = re.search(r'/(\d{14})(?:if_|mp_)*/https*://(.+/)', url).groups()
    except AttributeError:
        # There's something wrong with the link...
        # print(url)
        show_error(f'{url} isn\'t a Memento – did you forget to select an archive?')
    else:
        ss_file = Path('screenshots', f'{slugify(site)}-{date_str}-{save_width}.png')
        options = webdriver.FirefoxOptions()
        options.headless = True
        driver = webdriver.Firefox(executable_path=geckodriver, options=options)
        driver.implicitly_wait(30)
        driver.get(url)
        # Give some time for everything to load
        time.sleep(15)
        driver.maximize_window()
        current_width = driver.get_window_size()['width']
        # UK and AU use pywb in framed replay mode, so we need to switch to the framed content
        if domain in pywb:
            try:
                driver.switch_to.frame(0)
            except selenium.common.exceptions.NoSuchFrameException:
                show_error(f'Couldn\'t get a screenshot of {url} – sorry...')
                return
        try:
            ss = driver.find_element_by_tag_name('body').screenshot_as_base64
        except selenium.common.exceptions.NoSuchElementException:
            try:
                ss = driver.find_element_by_tag_name('frameset').screenshot_as_base64
            except selenium.common.exceptions.NoSuchElementException:
                show_error(f'Couldn\'t get a screenshot of {url} – sorry...')
                return
        driver.quit()
        img = Image.open(io.BytesIO(base64.b64decode(ss)))
        ratio = save_width / current_width
        (width, height) = (round(img.width * ratio), round(img.height * ratio))
        resized_img = img.resize((width, height), PIL.Image.LANCZOS)
        resized_img.save(ss_file)
        date = format_date_from_timestamp(url)
        status.clear_output()
        html_output.append(f'<div style="float:left; margin-left: 20px;"><p><b>{date}</b><br><a href="{url.replace("if_/", "/")}">{target_url.value}</a></p><p><a href="{ss_file}"><img src="{ss_file}"></a><br><a href="{ss_file}">[Download]</a></p></div>')
        with out:
            display((HTML(''.join(html_output))))

def show_error(message=None):
    status.clear_output()
    with status:
        print(f'Something went wrong – {message}')

def start(e):
    status.clear_output()
    out.clear_output(wait=True)
    with status:
        print('Generating screenshot...')
    if repository.value:
        memento = get_memento()
    else:
        memento = target_url.value
    if memento:
        get_full_page_screenshot(memento, save_width=width.value)
    else:
        show_error('couldn\'t find a Memento – sorry...')
    
def clear(e):
    global html_output
    html_output = []
    status.clear_output()
    out.clear_output()
    
def clear_last(e):
    global html_output
    html_output.pop()
    out.clear_output(wait=True)
    with out:
        display((HTML(''.join(html_output))))

repository = widgets.Dropdown(
    options=[('---', ''), ('UK Web Archive', 'bl'), ('National Library of Australia', 'nla'), ('National Library of New Zealand', 'nlnz'), ('Internet Archive', 'ia')],
    description='Archive:',
    disabled=False,
)

target_url = widgets.Text(description='Target URL:')

target_date = widgets.DatePicker(
    description='Target date: ',
    disabled=False
)

width = widgets.IntSlider(
    value=7,
    min=200,
    max=1000,
    step=100,
    description='Width:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

out = widgets.Output()
status = widgets.Output()
ss_button = widgets.Button(description='Get screenshot', button_style='primary')
ss_button.on_click(start)
clear_button = widgets.Button(description='Clear all')
clear_button.on_click(clear)
clear_last_button = widgets.Button(description='Clear last')
clear_last_button.on_click(clear_last)
note = '''
    <ul>
    <li>Select a repository, and insert a url to generate a screenshot from the archive.</li>
    <li>If you include a date, it\'ll attempt to find the closest capture using Memento Timegates.</li>
    <li>If you don't include a date, it'll give you the most recent capture.</li>
    <li>If you already have the url of the exact capture you want, just put it in the 'Target url' box and leave 'Archive' and 'Target date' blank.
    <li>You can add multiple screenshots to compare changes.</li>
    </ul>
    '''
display(HTML(note), widgets.HBox([widgets.VBox([repository, target_date]), widgets.VBox([target_url, width])], layout=widgets.Layout(padding='20px')), widgets.HBox([ss_button, clear_button, clear_last_button]), status, out)

----

Work on this notebook was supported by the [IIPC Discretionary Funding Programme 2019-2020](http://netpreserve.org/projects/)